In [27]:
!wget -O movielens-100k.zip http://files.grouplens.org/datasets/movielens/ml-100k.zip
!unzip -q movielens-100k.zip -d movielens-100k

--2023-05-08 05:33:24--  http://files.grouplens.org/datasets/movielens/ml-100k.zip
Resolving files.grouplens.org (files.grouplens.org)... 128.101.65.152
Connecting to files.grouplens.org (files.grouplens.org)|128.101.65.152|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 4924029 (4.7M) [application/zip]
Saving to: ‘movielens-100k.zip’

movielens-100k.zip  100%[===================>]   4.70M  27.4MB/s    in 0.2s    

2023-05-08 05:33:24 (27.4 MB/s) - ‘movielens-100k.zip’ saved [4924029/4924029]

replace movielens-100k/ml-100k/allbut.pl? [y]es, [n]o, [A]ll, [N]one, [r]ename: A


In [43]:
# Import necessary libraries
import numpy as np
import pandas as pd
from numpy.linalg import svd

# Load Movielens 100K dataset
url = 'http://files.grouplens.org/datasets/movielens/ml-100k/u1.base'
train_data = pd.read_csv(url, sep='\t', header=None, names=['user_id', 'item_id', 'rating', 'timestamp'])

url = 'http://files.grouplens.org/datasets/movielens/ml-100k/u1.test'
test_data = pd.read_csv(url, sep='\t', header=None, names=['user_id', 'item_id', 'rating', 'timestamp'])


In [44]:
# Convert data into user-item rating matrix
train_matrix = np.zeros((943, 1682))
for row in train_data.itertuples():
    train_matrix[row[1]-1, row[2]-1] = row[3]

test_matrix = np.zeros((943, 1682))
for row in test_data.itertuples():
    test_matrix[row[1]-1, row[2]-1] = row[3]


In [52]:
# Define function for Schatten-p norm minimization
def schatten_p_norm_minimization(matrix, p, lmbda, num_iters):
    for i in range(num_iters):
        U, S, Vt = svd(matrix, full_matrices=False)
        S = np.sign(S) * np.maximum(np.abs(S) - lmbda * p, 0)
        matrix = U @ np.diag(S) @ Vt
    return matrix

# Define function to calculate NMAE
def calculate_nmae(actual, predicted):
    mask = actual > 0
    nmae = np.mean(np.abs(actual[mask] - predicted[mask]) / np.max(actual))
    return nmae

In [ ]:
# Perform Schatten-p norm minimization and calculate NMAE for p = 0.1, p = 0.5, and p = 1
for p in [0.1, 0.5, 1]:
    reconstructed_matrix = schatten_p_norm_minimization(train_matrix, p, 0.2, 100)
    nmae = calculate_nmae(test_matrix, reconstructed_matrix)
    print(f'NMAE for p = {p}: {nmae:f}')

NMAE for p = 0.1: 0.698824
NMAE for p = 0.5: 0.665993
